In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
from keras.models import load_model
from keras.utils import to_categorical
import math
import pickle
from sklearn.preprocessing import StandardScaler


In [5]:
index = "dowjones"
# index = "frankfurt"
with open(f"../data/{index}_calculated/periods750_250_240.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
timestep = 240

In [7]:
def long_short_postion(returns, k):
    position_ = np.copy(returns)
    short = np.argpartition(position_, k)[:k]
    neutral = np.argpartition(position_, len(position_) - k)[:(len(position_) - k)]
    position_[:] = 1
    position_[neutral] = 0
    position_[short] = -1
    return position_

In [8]:
positions = []
for i in range(len(dataset[0])):
    model_period = f"../model/LSTM/{index}2_period{i}.h5"
    regressor = load_model(model_period, custom_objects={"softmax": softmax})
    x_train = dataset[0][i][0].values
    scaler = StandardScaler().fit(x_train)
    
    x_test = scaler.transform(dataset[1][i][0])
    y_test = to_categorical(dataset[1][i][1].values, 2)
#     print(f"Period {i}")
#     print(f"x test shape: {x_test.shape}")
#     print(f"y test shape: {y_test.shape}")
#     print(f"x_final shape: {x_final.shape}")
#     print(f"y_final shape: {y_final.shape}")
    x_series = [x_test[i:i + timestep, :]
                for i in range(x_test.shape[0] - timestep)]
    y_series = [y_test[i + timestep]
                for i in range(y_test.shape[0] - timestep)]
    x_final = np.array(x_series)
    y_final = np.array(y_series)



    predicted =  regressor.predict(x_final)
    label = predicted > 0.5
    label = label * 1 # Convert boolean to int
#     print((sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum())
    positions.append(predicted[:, :, 1])

ValueError: Error when checking input: expected lstm_10_input to have shape (240, 1) but got array with shape (240, 31)

In [ ]:
stocks = pd.read_csv("../data/dowjones/all_stocks_2006-01-01_to_2018-01-01.csv", index_col='Date',parse_dates=['Date'])
stocks = stocks[["Close", "Name"]]
stocks = stocks.pivot_table(values='Close', index=stocks.index, columns='Name', aggfunc='first')
stocks.head()

In [ ]:
real_returns = (stocks - stocks.shift(1)) / stocks.shift(1)
real_returns = real_returns.dropna()
real_returns[750:3000].head()

In [ ]:
all_positions = np.concatenate(positions,axis=0)
all_positions.shape

In [ ]:
for k in range(1, 16):
    probabilities = pd.DataFrame(data=all_positions, index=real_returns[750:3000].index, columns=real_returns.columns)
    position = probabilities.apply(lambda x: long_short_postion(x, k), axis=1, result_type='broadcast')
    final_returns = real_returns[750:3000].mul(position)
#     no_rebalance = (final_returns + 1).product().sum()/(2 * k)
#     rebalance = (1 + final_returns.sum(axis = 1)/k).product()
    rebalance1 = (1 + final_returns.sum(axis = 1)/(2 * k)).product()
#     print(f"no rebalance = {no_rebalance}")
#     print(f"rebalance = {rebalance}")
    print(f"rebalance = {rebalance1}")